In [1]:
import pandas as pd
from konlpy.tag import Okt
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np


###### 필요한 함수 정의 ######

def stemmer(text):
    text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]","",text)
    okt = Okt()
    text = okt.morphs(text, stem= True)
    return text

def remove_stop_words(text):
    word_token = stemmer(text)
    stop_words = pd.read_csv("data/korean_stop_words.txt")
    result = [word for word in word_token if not word in stop_words]
    return result

def create_dic(texts, num):
    tk = tf.keras.preprocessing.text.Tokenizer(num_words = num, oov_token = '알 수 없음')
    
    # 빈도순으로 순위 인덱스 매긴 단어 목록 생성
    tk.fit_on_texts(texts)
    word_dic = tk.word_index
    
    return word_dic



####################################

# 데이터 불러오기 (0412 뉴스데이터)
data = pd.read_csv('data/2022-04-12 news data, test.csv')

# 714번째 결측치 제거
df = data.drop([714], axis = 0)
df = df.reset_index()

title = df['기사제목']

clean_title = []
for i in range(len(title)):
    clean_title.append(remove_stop_words(title[i]))

# 이중 리스트 1차원으로 변환
clean_title = sum(clean_title, [])

# 단어사전 만들기
title_dic = create_dic(clean_title, 1000)


########## 전처리 관련 시도 ##########
okt = Okt()

tokenized_title = okt.pos(title[1045])
tokenized_nouns = ' '.join([word[0] for word in tokenized_title if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_title[:10])
print('명사 추출 :',tokenized_nouns)


품사 태깅 10개만 출력 : [('마크', 'Noun'), ('롱', 'Noun'), (',', 'Punctuation'), ('우', 'Adverb'), ('크라', 'Verb'), ('올인', 'Noun'), ('한', 'Josa'), ('사이', 'Noun'), ('…', 'Punctuation'), ('르펜', 'Noun')]
명사 추출 : 마크 롱 올인 사이 르펜 문제 맹 추격


In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def show_relevant_keyword(keyword, df): # 키워드와 사용할 뉴스 데이터를 인자로 입력받음
    
    # stop_words를 전에 했던 것처럼 txt 파일 형태로 제공하면 에러 발생!
    # 일단 에러를 막기 위해 'english'로 설정하였음. 
    # max_features는 임의로 2000개로 설정하였음.
    tv = TfidfVectorizer(stop_words = 'english', max_features = 2000)
    x = tv.fit_transform(df.기사제목)
    # words에는 feature가 된 단어들이 2000개 담겨 있음. 
    words = tv.get_feature_names() 

    ############## SVD 특이값 분해 ################
    from sklearn.decomposition import TruncatedSVD

    # n_components 는 max_features보다 적어야 함. 
    # 임의로 300개로 설정하였음
    svd = TruncatedSVD(n_components = 300, random_state = 1234)
    word_idx = words.index(keyword) # 사용자에게 입력받은 키워드의 인덱스 위치 확인
    svd.fit(x)

    # 시각화 
    # import matplotlib.pyplot as plt
    # plt.plot(svd.components_[:, word_idx])

    kw_idx = svd.components_[:, word_idx].argmax()

    relevant_words_df = pd.DataFrame({'단어': words, 'loading': svd.components_[kw_idx]})
    rel_words_df = relevant_words_df.sort_values('loading').tail()
    rel_words_df = rel_words_df.sort_values('loading', ascending = False)

    # 사용자가 입력한 키워드와 관련있는 상위 5개 단어를 데이터 프레임으로 반환
    return rel_words_df
    


In [3]:
show_relevant_keyword('윤석열', df)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,단어,loading
1226,윤석열,0.317788
538,박근혜,0.280170
1008,오늘,0.253666
392,단독,0.253287
641,사망,0.223976


: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 